# Script to check calibration of PBPS133
### Purpose: Short script scan PBPS133 positions once calibrated to check in databuffer
Contact christopher.arrell@psi.ch

In [4]:
import epics as ep
from bsread import source
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
from bsread import source
from IPython.display import clear_output, display
from scipy.optimize import curve_fit
import json
%matplotlib inline

## Setup and functions setup

In [4]:
channels = ['SAROP11-CVME-PBPS2:ght','Left']
Devive_prefix = "SAROP21-PBPS133:"

PBPS_x_PV_name = Devive_prefix+'MOTOR_X1.VAL'
PBPS_y_PV_name = Devive_prefix+'MOTOR_Y1.VAL'

PBPS_x_PV = ep.PV(PBPS_x_PV_name)
PBPS_y_PV = ep.PV(PBPS_y_PV_name)


def set_PBPS_x(pos):
    PBPS_x_PV.put(pos, wait=True)
def set_PBPS_y(pos):
    PBPS_y_PV.put(pos, wait=True)

## User inputs

In [3]:
waittime = 10 # in seconds
Scan_x_range = np.linspace(-0.15,0.15,10)
Scan_y_range = np.linspace(-0.15,0.15,10)

## Test scans
### The purpose of the cells bellow is to scan the PBPS with the calibrated values.  The results can be seen in the data buffer ui here:
[Vertical](https://ui-data-api.psi.ch/loadhash/2021-03-17/1da61ec6433d9726cd76fb92a5ce6c7707429aa1815915891bf40f124f529bc6)  
[Horizontal](https://ui-data-api.psi.ch/loadhash/2021-03-17/123ca6c0670638904964d091a7f4df1998f06437c06eab9eaf9190db8448384f)

In [7]:
for pos in Scan_y_range:
    set_PBPS_y(pos)
    sleep(waittime)
set_PBPS_y(0)

In [6]:
for pos in Scan_x_range:
    set_PBPS_x(pos)
    sleep(waittime)
set_PBPS_x(0)

## Get calibration from IOC and push to pipeline

In [1]:
config_fn = "/sf/photo/pipeline_cam_server/PBPS133/SAROP21-PBPS133_proc.json"
with open(config_fn, "r") as jsonFile:
    data = json.load(jsonFile)
data
data["right_calib"] = ep.caget('SLAAR21-LTIM01-EVR0:CALCX.E')
data["left_calib"] = ep.caget('SLAAR21-LTIM01-EVR0:CALCX.F')
data["up_calib"] = ep.caget('SLAAR21-LTIM01-EVR0:CALCY.F')
data["down_calib"] = ep.caget('SLAAR21-LTIM01-EVR0:CALCY.E')
data["vert_calib"] = ep.caget('SLAAR21-LTIM01-EVR0:CALCY.I')
data["horiz_calib"] = ep.caget('SLAAR21-LTIM01-EVR0:CALCX.I')

with open(config_fn, "w") as jsonFile:
    json.dump(data, jsonFile, indent=4)

NameError: name 'json' is not defined

In [6]:
! python /sf/photo/pipeline_cam_server/PBPS133/update.py

Traceback (most recent call last):
  File "/sf/photo/pipeline_cam_server/PBPS133/update.py", line 10, in <module>
    with open("SAROP21-PBPS133_proc.json") as config_file:
FileNotFoundError: [Errno 2] No such file or directory: 'SAROP21-PBPS133_proc.json'
